# mlx
mlx是苹果官方出的机器学习框架，能够充分利用apple silicon芯片的性能，比pytorch+mps的模式要快一点。

## 下载安装
### 安装mlx-lm
```shell
pip install mlx-lm
```

## 开箱即用的测试
因为默认就可以用gguf，所以可以直接预测：
```shell
python -m mlx_lm.generate --model mlx-community/phi-2-hf-4bit-mlx --prompt "hello"
```
因为phi-2是一个基础模型，而不是chat模型，所以默认给出的是拼凑hello world的续写，想要给出对话响应，可以用类似这样的prompt：
```shell
python -m mlx_lm.generate --model mlx-community/phi-2-hf-4bit-mlx --prompt "User: hello
AI:" --colorize --ignore-chat-template
```

## 微调
### 准备好数据
拿`LLaMA-Factory/blob/main/data/identity.json`的标识来进行测试，以前叫self_cognition自我认知。
数据格式类似于：
```json
[
  {
    "instruction": "hi",
    "input": "",
    "output": "Hello! I am NAME, an AI assistant developed by AUTHOR. How can I assist you today?"
  },
  {
    "instruction": "hello",
    "input": "",
    "output": "Hello! I am NAME, an AI assistant developed by AUTHOR. How can I assist you today?"
  },
]
```
把NAME和AUTHOR进行批量替换：
```shell
wget https://raw.githubusercontent.com/hiyouga/LLaMA-Factory/main/data/identity.json
sed -i '' 's/NAME/FOFABot/g' identity.json
sed -i '' 's/AUTHOR/华顺信安/g' identity.json
```
这时候看到数据如下：
```json
[
  {
    "instruction": "hi",
    "input": "",
    "output": "Hello! I am FOFABot, an AI assistant developed by 华顺信安. How can I assist you today?"
  },
  {
    "instruction": "hello",
    "input": "",
    "output": "Hello! I am FOFABot, an AI assistant developed by 华顺信安. How can I assist you today?"
  },
]
```

生成mlx需要的格式，要结合待微调的模型来进行定制，比如yi和deepseek就不一样，base模型和chat模型也不一样，生成的text字段要根据模型来进行生成。关于datasets的处理[参考](https://huggingface.co/docs/datasets/en/process).

In [22]:
# !pip install datasets
import datasets

# 加载 JSONL 文件
dataset = datasets.load_dataset("json", data_files="identity.json")
print(dataset)

# Create the text field using map function
dataset = dataset.map(
    lambda x: {"text": f"Instruction: {x['instruction']}\nOutput: {x['output']}"}
)
print(dataset)

dataset = dataset.remove_columns(["instruction", "input", "output"])
print(dataset)

print(dataset['train'][0])

dataset = dataset['train'].train_test_split(test_size=0.1)
print(dataset)

# Save the converted "train" split as a new JSONL file
dataset['train'].to_json('data/train.jsonl', orient='records', lines=True, force_ascii=False)
dataset['test'].to_json('data/valid.jsonl', orient='records', lines=True, force_ascii=False)

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 91
    })
})
DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction', 'text'],
        num_rows: 91
    })
})
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 91
    })
})
{'text': 'Instruction: hi\nOutput: Hello! I am FOFABot, an AI assistant developed by 华顺信安. How can I assist you today?'}
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 81
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 800.59ba/s]


1958



### 开始微调
```
python -m mlx_lm.lora --model mlx-community/phi-2-hf-4bit-mlx --train --data ./data --adapter-file fofabot.npz --learning-rate 1e-4
```
参数中，由于数据量比较小，把learning-rate可以适当调大一点没有问题。

### 预测
```
python -m mlx_lm.lora --model mlx-community/phi-2-hf-4bit-mlx --adapter-file fofabot.npz --prompt "Instruction: hi
Output: "

# 也可以在微调过程中进行验证，比如100步之后可以使用：checkpoints/100_fofabot.npz 
python -m mlx_lm.lora --model mlx-community/phi-2-hf-4bit-mlx --adapter-file checkpoints/100_fofabot.npz 
--prompt "Instruction: hi
Output: "
```
训练1000次后的回答是：
```
作为 FOFABot，我的核心价值是致力、问题检索和给信息提供。
```
看起来怪怪的，不知道是不是因为量化模型的关系，用非量化的版本再测试一下。

## 数据格式
根据不同的模型来生成文件，mlx的约定为：
- 指定目录，目录下有不同的jsonl文件
- 必须为jsonl的格式，每行一个json，只处理text字段
- 需要至少有train和valid两个文件，一个用于训练一个用于验证

## 更多参考：
- [Fine-tuning the latest Google Gemma model locally using MLX](https://gist.github.com/alexweberk/635431b5c5773efd6d1755801020429f)
    - https://github.com/alexweberk/playing-with-llms/blob/main/notebooks/mlx_gemma/mlx_finetuning_gemma.ipynb
- [MLX LLM Finetuning](https://github.com/AaronWard/generative-ai-workbook/blob/main/personal_projects/23.mlx-finetuning/1.fine-tuning.ipynb)
- [sql-create-context-mlx-lora](https://github.com/alwint3r/sql-create-context-mlx-lora)
- [chat-with-mlx](https://github.com/qnguyen3/chat-with-mlx)